# **SETUP**

### **Imports**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 37.3 MB/s 
     |████████████████████████████████| 163 kB 59.1 MB/s 


In [3]:
import os
import math
import argparse
import sys
import random
import pickle
import spacy
import numpy as np

In [4]:
from time import strftime, localtime
from transformers import BertTokenizer
from sklearn import metrics
from xml.etree.ElementTree import parse
from spacy.tokens import Doc

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

# **CLASSES**

### **Data**

In [6]:
def pad_and_truncate(sequence, max_len, dtype='int64', padding='post', truncating='post', value=0):
    x = (np.ones(max_len) * value).astype(dtype)
    if truncating == 'pre':
        trunc = sequence[-max_len:]
    else:
        trunc = sequence[:max_len]
    trunc = np.asarray(trunc, dtype=dtype)
    if padding == 'post':
        x[:len(trunc)] = trunc
    else:
        x[-len(trunc):] = trunc
    return x

In [7]:
class Tokenizer():
    def __init__(self, max_len, lower=True):
        self.lower = lower
        self.max_len = max_len
        self.word_to_index = {}
        self.index_to_word = {}
        self.idx = 1

    def fit_on_text(self, text):
        if self.lower:
            text = text.lower()
        words = text.split()
        for word in words:
            if word not in self.word_to_index:
                self.word_to_index[word] = self.idx
                self.index_to_word[self.idx] = word
                self.idx += 1

    def text_to_sequence(self, text, reverse=False, padding='post', truncating='post'):
        if self.lower:
            text = text.lower()
        words = text.split()
        unk = len(self.word_to_index)+1
        sequence = [self.word_to_index[w] if w in self.word_to_index else unk for w in words]
        if len(sequence) == 0:
            sequence = [0]
        if reverse:
            sequence = sequence[::-1]
        return pad_and_truncate(sequence, self.max_len, padding=padding, truncating=truncating)

In [8]:
class ABSADataset(Dataset):
    def __init__(self, file_name, tokenizer):
        with open(file_name, 'r', newline='\n', encoding='utf-8', errors='ignore') as f:
            lines = f.readlines()
        with open(file_name + '.graph', 'rb') as f:
            index_to_graph = pickle.load(f)
        
        self.data = []

        # Create tokens with tokenizer
        for i in range(0, len(lines), 3):
            left, right = [s.lower().strip() for s in lines[i].split("$T$")]
            aspect = lines[i+1].lower().strip()
            polarity = lines[i+2].strip()

            text_token = tokenizer.text_to_sequence(" ".join([left, aspect, right]))
            context_token = tokenizer.text_to_sequence(" ".join([left, right]))

            left_token = tokenizer.text_to_sequence(left)
            left_aspect_token = tokenizer.text_to_sequence(" ".join([left, aspect]))

            right_token = tokenizer.text_to_sequence(right, reverse=True)
            right_aspect_token = tokenizer.text_to_sequence(" ".join([aspect, right]), reverse=True)

            aspect_token = tokenizer.text_to_sequence(aspect)

            left_len = np.sum(left_token != 0)
            aspect_len = np.sum(aspect_token != 0)
            aspect_boundary = np.asarray([left_len, left_len + aspect_len - 1], dtype = np.int64)
            polarity = int(polarity) + 1
            
            text_len = np.sum(text_token != 0)
            concat_segments_tokens = pad_and_truncate([0] * (text_len + 1) + [1] * (aspect_len + 1), tokenizer.max_len)
            
            dependency_graph = np.pad(index_to_graph[i],
                                      ((0, tokenizer.max_len-index_to_graph[i].shape[0]),
                                       (0, tokenizer.max_len-index_to_graph[i].shape[0])),
                                      'constant')
            
            self.data.append({
                'text_token' : text_token,
                'context_token' : context_token, 
                'left_token' : left_token, 
                'left_aspect_token' : left_aspect_token,
                'right_token' : right_token,
                'right_aspect_token' : right_aspect_token,
                'aspect_token' : aspect_token, 
                'aspect_boundary' : aspect_boundary, 
                'polarity' : polarity, 
                'dependency_graph' : dependency_graph
            })
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)

In [9]:
class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split()
        # All tokens 'own' a subsequent space character in this tokenizer
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

### **Model**

In [10]:
class LSTM(nn.Module):
    def __init__(self, embedding_matrix):
        super(LSTM, self).__init__()
        self.embed = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float))
        self.squeeze_embedding = SqueezeEmbedding()
        self.lstm = nn.LSTM(config2["embed_dim"]*2, config2["hidden_dim"], num_layers=1, batch_first=True)
        self.dense = nn.Linear(config2["hidden_dim"], config2["polarities_dim"])
    
    def forward(self, inputs):
        text_indices, aspect_indices = inputs[0], inputs[1]
        x_len = torch.sum(text_indices != 0, dim=-1)
        x_len_max = torch.max(x_len)
        aspect_len = torch.sum(aspect_indices != 0, dim=-1).float()

        x = self.embed(text_indices)
        x = self.squeeze_embedding(x, x_len)
        aspect = self.embed(aspect_indices)
        aspect_pool = torch.div(torch.sum(aspect, dim=1), aspect_len.unsqueeze(1))
        aspect = aspect_pool.unsqueeze(1).expand(-1, x_len_max, -1)
        x = torch.cat((aspect, x), dim=-1)

        x_sort_idx = torch.sort(-x_len)[1].long()
        x_unsort_idx = torch.sort(x_sort_idx)[1].long()
        x_len = x_len[x_sort_idx]
        x = x[x_sort_idx]

        x_emb_p = torch.nn.utils.rnn.pack_padded_sequence(x, x_len, batch_first=True)

        global ht
        out_pack, (ht, ct) = self.lstm(x_emb_p, None)
        # print(out_pack)
        # print(out_pack.shape)

        # ht = torch.transpose(ht, 0, 1)[
        #     x_unsort_idx]  # (num_layers * num_directions, batch, hidden_size) -> (batch, ...)
        # ht = torch.transpose(ht, 0, 1)

        # """unpack: out"""
        # out = torch.nn.utils.rnn.pad_packed_sequence(out_pack, batch_first=True)  # (sequence, lengths)
        # out = out[0]  #
        # # global out
        # out = out[x_unsort_idx]

        return self.dense(ht[0])

In [11]:
class SqueezeEmbedding(nn.Module):
    """
    Squeeze sequence embedding length to the longest one in the batch
    """
    def __init__(self, batch_first=True):
        super(SqueezeEmbedding, self).__init__()
        self.batch_first = batch_first

    def forward(self, x, x_len):
        """
        sequence -> sort -> pad and pack -> unpack ->unsort
        :param x: sequence embedding vectors
        :param x_len: numpy/tensor list
        :return:
        """
        """sort"""
        x_sort_idx = torch.sort(-x_len)[1].long()
        x_unsort_idx = torch.sort(x_sort_idx)[1].long()
        x_len = x_len[x_sort_idx]
        x = x[x_sort_idx]
        """pack"""
        x_emb_p = torch.nn.utils.rnn.pack_padded_sequence(x, x_len.cpu(), batch_first=self.batch_first)
        """unpack: out"""
        out = torch.nn.utils.rnn.pad_packed_sequence(x_emb_p, batch_first=self.batch_first)  # (sequence, lengths)
        out = out[0]  #
        """unsort"""
        out = out[x_unsort_idx]
        return out

### **Main**

In [12]:
class Instructor:
    def __init__(self):
        tokenizer = build_tokenizer(
            fnames=[config2["dataset_file"]['train'], config2["dataset_file"]['test']],
            max_seq_len=config2["max_seq_len"],
            dat_fname='{0}_tokenizer.dat'.format(config2["dataset"]))
        embedding_matrix = build_embedding_matrix(
            word2idx=tokenizer.word_to_index,
            embed_dim=config2["embed_dim"],
            dat_fname='{0}_{1}_embedding_matrix.dat'.format(str(config2["embed_dim"]), config2["dataset"]))
        self.model = config2["model_class"](embedding_matrix).to(config2["device"])

        self.trainset = ABSADataset(config2["dataset_file"]['train'], tokenizer)
        self.testset = ABSADataset(config2["dataset_file"]['test'], tokenizer)
        assert 0 <= config2["valset_ratio"] < 1
        if config2["valset_ratio"] > 0:
            valset_len = int(len(self.trainset) * config2["valset_ratio"])
            self.trainset, self.valset = random_split(self.trainset, (len(self.trainset)-valset_len, valset_len))
        else:
            self.valset = self.testset

        if config2["device"].type == 'cuda':
            print('cuda memory allocated: {}'.format(torch.cuda.memory_allocated(device=config2["device"].index)))
        self._print_args()

    def _print_args(self):
        n_trainable_params, n_nontrainable_params = 0, 0
        for p in self.model.parameters():
            n_params = torch.prod(torch.tensor(p.shape))
            if p.requires_grad:
                n_trainable_params += n_params
            else:
                n_nontrainable_params += n_params
        print('> n_trainable_params: {0}, n_nontrainable_params: {1}'.format(n_trainable_params, n_nontrainable_params))


    def _reset_params(self):
        for child in self.model.children():
            for p in child.parameters():
                if p.requires_grad:
                    if len(p.shape) > 1:
                        config2["initializer"](p)
                    else:
                        stdv = 1. / math.sqrt(p.shape[0])
                        torch.nn.init.uniform_(p, a=-stdv, b=stdv)

    def _train(self, criterion, optimizer, train_data_loader, val_data_loader):
        max_val_acc = 0
        max_val_f1 = 0
        max_val_epoch = 0
        global_step = 0
        path = None
        for i_epoch in range(config2["num_epoch"]):
            print('>' * 100)
            print('epoch: {}'.format(i_epoch))
            n_correct, n_total, loss_total = 0, 0, 0
            # switch model to training mode
            self.model.train()
            for i_batch, batch in enumerate(train_data_loader):
                global_step += 1
                # clear gradient accumulators
                optimizer.zero_grad()

                inputs = [batch[col].to(config2["device"]) for col in config2["inputs_cols"]]
                global outputs, targets
                outputs = self.model(inputs)
                targets = batch['polarity'].to(config2["device"])
                # print(outputs)
                # print(targets)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                n_correct += (torch.argmax(outputs, -1) == targets).sum().item()
                n_total += len(outputs)
                loss_total += loss.item() * len(outputs)
                if global_step % config2["log_step"] == 0:
                    train_acc = n_correct / n_total
                    train_loss = loss_total / n_total
                    print('[epoch {}] loss: {:.4f}, acc: {:.4f}'.format(i_epoch, train_loss, train_acc))

            val_acc, val_f1 = self._evaluate_acc_f1(val_data_loader)
            print('> val_acc: {:.4f}, val_f1: {:.4f}'.format(val_acc, val_f1))
            if val_acc > max_val_acc:
                max_val_acc = val_acc
                max_val_epoch = i_epoch
                if not os.path.exists('state_dict'):
                    os.mkdir('state_dict')
                path = '{0}/{1}_{2}_val_acc_{3}'.format(config["model_path"], config2["model_name"], config2["dataset"], round(val_acc, 4))
                torch.save(self.model.state_dict(), path)
                print('>> saved: {}'.format(path))
            if val_f1 > max_val_f1:
                max_val_f1 = val_f1
            if i_epoch - max_val_epoch >= config2["patience"]:
                print('>> early stop.')
                break

        return path

    def _evaluate_acc_f1(self, data_loader):
        n_correct, n_total = 0, 0
        t_targets_all, t_outputs_all = None, None
        # switch model to evaluation mode
        self.model.eval()
        with torch.no_grad():
            for i_batch, t_batch in enumerate(data_loader):
                t_inputs = [t_batch[col].to(config2["device"]) for col in config2["inputs_cols"]]
                t_targets = t_batch['polarity'].to(config2["device"])
                t_outputs = self.model(t_inputs)

                n_correct += (torch.argmax(t_outputs, -1) == t_targets).sum().item()
                n_total += len(t_outputs)

                if t_targets_all is None:
                    t_targets_all = t_targets
                    t_outputs_all = t_outputs
                else:
                    t_targets_all = torch.cat((t_targets_all, t_targets), dim=0)
                    t_outputs_all = torch.cat((t_outputs_all, t_outputs), dim=0)

        acc = n_correct / n_total
        f1 = metrics.f1_score(t_targets_all.cpu(), torch.argmax(t_outputs_all, -1).cpu(), labels=[0, 1, 2], average='macro')
        return acc, f1

    def run(self):
        # Loss and Optimizer
        criterion = nn.CrossEntropyLoss()
        _params = filter(lambda p: p.requires_grad, self.model.parameters())
        optimizer = config2["optimizer"](_params, lr=config2["lr"], weight_decay=config2["l2reg"])

        train_data_loader = DataLoader(dataset=self.trainset, batch_size=config2["batch_size"], shuffle=True)
        test_data_loader = DataLoader(dataset=self.testset, batch_size=config2["batch_size"], shuffle=False)
        val_data_loader = DataLoader(dataset=self.valset, batch_size=config2["batch_size"], shuffle=False)

        self._reset_params()
        best_model_path = self._train(criterion, optimizer, train_data_loader, val_data_loader)
        self.model.load_state_dict(torch.load(best_model_path))
        test_acc, test_f1 = self._evaluate_acc_f1(test_data_loader)
        print('>> test_acc: {:.4f}, test_f1: {:.4f}'.format(test_acc, test_f1))

# **Functions**

In [13]:
def change_format_sentences(source_path, target_path):
    f = open(target_path, "w")

    sentences = parse(source_path).getroot()
    preprocessed = []

    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        aspectTerms = sentence.find('aspectTerms')
        if aspectTerms is None:
            continue
        for aspectTerm in aspectTerms:
            term = aspectTerm.get('term')
            polarity = aspectTerm.get('polarity')
            if polarity == "positive":
                polarity = '1'
            elif polarity == "neutral":
                polarity = '0'
            elif polarity == "negative":
                polarity = '-1'
            else:
                raise Exception("invalid polarity!")
            start = int(aspectTerm.get('from'))
            end = int(aspectTerm.get('to'))
            preprocessed.append(text[:start] + "$T$" + text[end:])
            preprocessed.append(text[start:end])
            preprocessed.append(polarity)
    f.write("\n".join(preprocessed))

In [14]:
def build_tokenizer(fnames, max_seq_len, dat_fname):
    if os.path.exists(dat_fname):
        print('loading tokenizer:', dat_fname)
        tokenizer = pickle.load(open(dat_fname, 'rb'))
    else:
        text = ''
        for fname in fnames:
            fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
            lines = fin.readlines()
            fin.close()
            for i in range(0, len(lines), 3):
                text_left, _, text_right = [s.lower().strip() for s in lines[i].partition("$T$")]
                aspect = lines[i + 1].lower().strip()
                text_raw = text_left + " " + aspect + " " + text_right
                text += text_raw + " "

        tokenizer = Tokenizer(max_seq_len)
        tokenizer.fit_on_text(text)
        pickle.dump(tokenizer, open(dat_fname, 'wb'))
    return tokenizer

In [15]:
def _load_word_vec(path, word2idx=None, embed_dim=300):
    fin = open(path, 'r', encoding='utf-8', newline='\n', errors='ignore')
    word_vec = {}
    for line in fin:
        tokens = line.rstrip().split()
        word, vec = ' '.join(tokens[:-embed_dim]), tokens[-embed_dim:]
        if word in word2idx.keys():
            word_vec[word] = np.asarray(vec, dtype='float32')
    return word_vec

In [16]:
def build_embedding_matrix(word2idx, embed_dim, dat_fname):
    if os.path.exists(dat_fname):
        print('loading embedding_matrix:', dat_fname)
        embedding_matrix = pickle.load(open(dat_fname, 'rb'))
    else:
        print('loading word vectors...')
        embedding_matrix = np.zeros((len(word2idx) + 2, embed_dim))  # idx 0 and len(word2idx)+1 are all-zeros
        fname = config["glove_path"]
        word_vec = _load_word_vec(fname, word2idx=word2idx, embed_dim=embed_dim)
        print('building embedding_matrix:', dat_fname)
        for word, i in word2idx.items():
            vec = word_vec.get(word)
            if vec is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = vec
        pickle.dump(embedding_matrix, open(dat_fname, 'wb'))
    return embedding_matrix

In [17]:
def pad_and_truncate(sequence, maxlen, dtype='int64', padding='post', truncating='post', value=0):
    x = (np.ones(maxlen) * value).astype(dtype)
    if truncating == 'pre':
        trunc = sequence[-maxlen:]
    else:
        trunc = sequence[:maxlen]
    trunc = np.asarray(trunc, dtype=dtype)
    if padding == 'post':
        x[:len(trunc)] = trunc
    else:
        x[-len(trunc):] = trunc
    return x

In [18]:
def dependency_adj_matrix(text):
    # https://spacy.io/docs/usage/processing-text
    tokens = nlp(text)
    words = text.split()
    matrix = np.zeros((len(words), len(words))).astype('float32')
    assert len(words) == len(list(tokens))

    for token in tokens:
        matrix[token.i][token.i] = 1
        for child in token.children:
            matrix[token.i][child.i] = 1
            matrix[child.i][token.i] = 1

    return matrix

In [19]:
def process_data():
    change_format_sentences(config["raw_train_path"], config["processed_train_path"])
    change_format_sentences(config["raw_val_path"], config["processed_val_path"])
    change_format_sentences(config["raw_test_path"], config["processed_test_path"])
    process(config["processed_train_path"])
    process(config["processed_val_path"])
    process(config["processed_test_path"])

In [20]:
def process(filename):
    fin = open(filename, 'r', encoding='utf-8', newline='\n', errors='ignore')
    lines = fin.readlines()
    fin.close()
    idx2graph = {}
    fout = open(filename+'.graph', 'wb')
    for i in range(0, len(lines), 3):
        text_left, _, text_right = [s.strip() for s in lines[i].partition("$T$")]
        aspect = lines[i + 1].strip()
        adj_matrix = dependency_adj_matrix(text_left+' '+aspect+' '+text_right)
        idx2graph[i] = adj_matrix
    pickle.dump(idx2graph, fout)        
    fout.close() 

In [21]:
def run():
    ins = Instructor()
    ins.run()

# **MAIN**

### **Configuration**

In [22]:
nlp = spacy.load('en_core_web_sm')
nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

In [23]:
config = {
    "base_path": "/content/drive/MyDrive/CS4248/MAMS-ATSA",
    "glove_path": "/content/drive/MyDrive/CS4248/glove.42B.300d.txt"
}

config["model_path"] = os.path.join(config["base_path"], 'lstm_models')
config["raw_train_path"] = os.path.join(config["base_path"], 'raw/train.xml')
config["raw_val_path"] = os.path.join(config["base_path"], 'raw/val.xml')
config["raw_test_path"] = os.path.join(config['base_path'], 'raw/test.xml')
config["processed_train_path"] = os.path.join(config["base_path"], 'raw/train_processed.xml.seg')
config["processed_val_path"] = os.path.join(config["base_path"], 'raw/val_processed.xml.seg')
config["processed_test_path"] = os.path.join(config['base_path'], 'raw/test_processed.xml.seg')

In [24]:
dataset_files = {
    'train': config["processed_train_path"],
    'test': config["processed_test_path"]
}

config2 = {
    "model_name" : "LSTM",
    "lr" : 2e-5,
    "dropout" : 0.1,
    "l2reg" : 0.01,
    "num_epoch" : 20,
    "batch_size" : 16,
    "log_step" : 10,
    "embed_dim" : 300,
    "hidden_dim" : 300,
    "model_class" : LSTM,
    "dataset": "MAMS",
    "dataset_file" : dataset_files,
    "inputs_cols" : ['text_token', 'aspect_token'],
    "initializer" : torch.nn.init.xavier_uniform_,
    "optimizer" : torch.optim.Adam,
    "max_seq_len" : 85,
    "polarities_dim" : 3,
    "hops" : 3,
    "patience" : 5,
    "device" : None,
    "seed" : 1234,
    "valset_ratio" : 0
}

config2["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu') \
        if config2["device"] is None else torch.device(config2["device"])

if config2["seed"] is not None:
    random.seed(config2["seed"])
    np.random.seed(config2["seed"])
    torch.manual_seed(config2["seed"])
    torch.cuda.manual_seed(config2["seed"])
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(config2["seed"])

### **Pipeline**

In [25]:
#process_data()

In [26]:
run()

loading word vectors...
building embedding_matrix: 300_MAMS_embedding_matrix.dat
> n_trainable_params: 1083303, n_nontrainable_params: 3994500
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 0
[epoch 0] loss: 1.1148, acc: 0.3937
[epoch 0] loss: 1.1065, acc: 0.4000
[epoch 0] loss: 1.0862, acc: 0.4250
[epoch 0] loss: 1.0815, acc: 0.4297
[epoch 0] loss: 1.0800, acc: 0.4338
[epoch 0] loss: 1.0823, acc: 0.4344
[epoch 0] loss: 1.0775, acc: 0.4402
[epoch 0] loss: 1.0797, acc: 0.4344
[epoch 0] loss: 1.0833, acc: 0.4271
[epoch 0] loss: 1.0828, acc: 0.4288
[epoch 0] loss: 1.0782, acc: 0.4352
[epoch 0] loss: 1.0781, acc: 0.4365
[epoch 0] loss: 1.0816, acc: 0.4317
[epoch 0] loss: 1.0815, acc: 0.4313
[epoch 0] loss: 1.0804, acc: 0.4321
[epoch 0] loss: 1.0815, acc: 0.4297
[epoch 0] loss: 1.0814, acc: 0.4313
[epoch 0] loss: 1.0808, acc: 0.4326
[epoch 0] loss: 1.0803, acc: 0.4326
[epoch 0] loss: 1.0782, acc: 0.4359
[epoch 0] loss: 1.0765, acc